### Results Table

| Attack Technique   | Clean mAP@0.5 | Adversarial mAP@0.5 | Drop      |
|--------------------|---------------|---------------------|-----------|
| FGSM               | 0.927         | 0.312               | -0.615    |
| Fool Detectors     | 0.927         | 0.155               | -0.772    |
| UAP                | 0.927         | 0.443               | -0.484    |
| Disappearance DAG  | 0.935         | 0.547              | -0.388    |
| Targeted DAG       | 0.927         | 0.0947              | -0.8323   |



### getting a subset of the training data to apply attacks on
- applying the attacks on the full training data is not a good option as it will have a very high cost

In [1]:
!curl -L "https://universe.roboflow.com/ds/glquRaJDf9?key=1kWinBXVTQ" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/labels/001677_JPG_jpg.rf.414e120831f7e7f30447cb3cb26bdc83.txt  
 extracting: train/labels/001677_JPG_jpg.rf.d0e22a225cbb317066ae2e6efd3dbcc2.txt  
  inflating: train/labels/001678_JPG_jpg.rf.21782caf845bde6574cb51b3f5c74490.txt  
  inflating: train/labels/001678_JPG_jpg.rf.4de99d4836e879bc0a6b13d15eefe346.txt  
  inflating: train/labels/001678_JPG_jpg.rf.d9b987c7dd66459a58b74d90a6970de7.txt  
  inflating: train/labels/001679_JPG_jpg.rf.7f8083798d74d6e40543f776ec116c6a.txt  
  inflating: train/labels/001679_JPG_jpg.rf.c11f3850b8f5630a0b21810e0a256c90.txt  
  inflating: train/labels/001679_JPG_jpg.rf.f0cdb80eb57c2cf67c5306cbf7c19aac.txt  
  inflating: train/labels/001680_JPG_jpg.rf.05690ffb9e45dcb471a95f0b96806331.txt  
  inflating: train/labels/001680_JPG_jpg.rf.8472e670be11dc0438f4a3e001e124bb.txt  
  inflating: train/labels/001680_JPG_jpg.rf.fc6132ab57b715e2192ea08bbc90067d.txt  
  inflating: train/labels/001681_JPG

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.8/993.8 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [14]:
import os
import torch
import numpy as np
from sklearn.cluster import KMeans
from torchvision import models, transforms
from PIL import Image
from util import sample_images_by_class, SuppressPrints
from attacks import fgsm_attack_detector
import torch
from ultralytics import YOLO


In [17]:
# Step 1: Balance the dataset using your function.
images_dir = "/content/train/images"
labels_dir = "/content/train/labels"
num_per_class = 200

final_image_paths,labels_paths, dist = sample_images_by_class(images_dir, labels_dir, num_per_class)
print(f"Collected {len(final_image_paths)} images after balancing.")
for c, cnt in dist.items():
    print(f"Class {c}: {cnt}")


Collected 2762 images after balancing.
Class 13: 226
Class 11: 220
Class 8: 212
Class 0: 309
Class 12: 228
Class 1: 348
Class 14: 210
Class 9: 213
Class 7: 215
Class 4: 205
Class 6: 219
Class 10: 227
Class 3: 244
Class 5: 228
Class 2: 42


In [18]:
model = YOLO("yolov8n_TrafficSigns.pt")

In [23]:
import os
import torch
from torchvision.utils import save_image
from pathlib import Path
import shutil

# Output directories
output_image_dir = Path("fgsm_augmented_images/images")
output_label_dir = Path("fgsm_augmented_images/labels")
output_image_dir.mkdir(parents=True, exist_ok=True)
output_label_dir.mkdir(parents=True, exist_ok=True)

i = 0
# Attack loop
for img_path, label_path in zip(final_image_paths, labels_paths):
    img_path = Path(img_path)
    label_path = Path(label_path)

    with SuppressPrints():
        adv_tensor = fgsm_attack_detector(
            image_path=str(img_path),
            model_path="yolov8n_TrafficSigns.pt",
            epsilon=0.15,
            conf_threshold=0.25,
            device="cuda" if torch.cuda.is_available() else "cpu"
        )

    # Skip if attack failed
    if adv_tensor is None:
        continue

    base_name = f"adv_fgsm_{img_path.stem}"
    adv_image_path = output_image_dir / f"{base_name}.png"
    adv_label_path = output_label_dir / f"{base_name}.txt"

    save_image(adv_tensor.clone().detach().cpu(), adv_image_path, normalize=True)
    shutil.copy(label_path, adv_label_path)

    i += 1
    if i % 100 == 0:
        print(f"Generated {i} adversarial examples.")


Generated 100 adversarial examples.
Generated 200 adversarial examples.
Generated 300 adversarial examples.
Generated 400 adversarial examples.
Generated 500 adversarial examples.
Generated 600 adversarial examples.
Generated 700 adversarial examples.
Generated 800 adversarial examples.
Generated 900 adversarial examples.
Generated 1000 adversarial examples.
Generated 1100 adversarial examples.
Generated 1200 adversarial examples.
Generated 1300 adversarial examples.
Generated 1400 adversarial examples.
Generated 1500 adversarial examples.
Generated 1600 adversarial examples.
Generated 1700 adversarial examples.
Generated 1800 adversarial examples.
Generated 1900 adversarial examples.
Generated 2000 adversarial examples.
Generated 2100 adversarial examples.
Generated 2200 adversarial examples.
Generated 2300 adversarial examples.
Generated 2400 adversarial examples.
Generated 2500 adversarial examples.
Generated 2600 adversarial examples.
Generated 2700 adversarial examples.


In [24]:
import shutil
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
# Path to your folder and zip file
source_folder = 'fgsm_augmented_images'
zip_filename = 'fgsm_augmented_images.zip'

# Create the ZIP file
shutil.make_archive(base_name=zip_filename.replace('.zip', ''), format='zip', root_dir=source_folder)


'/content/fgsm_augmented_images.zip'

In [27]:
# Copy to Google Drive (change the path if needed)
drive_target = '/content/drive/MyDrive/yolo' + zip_filename
shutil.copy(zip_filename, drive_target)

print(f"✅ Zipped and copied to Drive: {drive_target}")

✅ Zipped and copied to Drive: /content/drive/MyDrive/yolofgsm_augmented_images.zip


In [30]:
!ls /content/drive/MyDrive/yolo

In [31]:
# prompt: merge output_image_dir and output_label_dir to "/content/train/images" and "/content/train/labels"

import shutil
from pathlib import Path

# Define source and destination directories
output_image_dir = Path("fgsm_augmented_images/images")
output_label_dir = Path("fgsm_augmented_images/labels")
destination_image_dir = "/content/train/images"
destination_label_dir = "/content/train/labels"

# Create destination directories if they don't exist
Path(destination_image_dir).mkdir(parents=True, exist_ok=True)
Path(destination_label_dir).mkdir(parents=True, exist_ok=True)

# Copy images
for image_file in output_image_dir.glob("*"):
    shutil.copy(image_file, destination_image_dir)

# Copy labels
for label_file in output_label_dir.glob("*"):
    shutil.copy(label_file, destination_label_dir)

print(f"Successfully merged images and labels to {destination_image_dir} and {destination_label_dir}")


Successfully merged images and labels to /content/train/images and /content/train/labels


In [37]:
model = YOLO("yolov8n_TrafficSigns.pt")


result = model.train(data='/content/data.yaml', epochs=10, batch=32)


Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n_TrafficSigns.pt, data=/content/data.yaml, epochs=10, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

train: Scanning /content/train/labels.cache... 13332 images, 15 backgrounds, 0 corrupt: 100%|██████████| 13332/13332 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/valid/labels.cache... 801 images, 0 backgrounds, 0 corrupt: 100%|██████████| 801/801 [00:00<?, ?it/s]


Plotting labels to runs/detect/train5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000526, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train5
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      10/10      5.57G     0.5524     0.4093     0.9571         26        640: 100%|██████████| 417/417 [04:01<00:00,  1.72it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:07<00:00,  1.82it/s]

                   all        801        944       0.96       0.89      0.956      0.819



10 epochs completed in 0.724 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 6.2MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,008,573 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:16<00:00,  1.26s/it]


                   all        801        944       0.96       0.89      0.956       0.82
           Green Light         87        122      0.903      0.639      0.818      0.485
             Red Light         74        108      0.878      0.611      0.822       0.49
       Speed Limit 100         52         52      0.912      0.942      0.986      0.884
       Speed Limit 110         17         17      0.902          1      0.979      0.922
       Speed Limit 120         60         60          1      0.904      0.993      0.901
        Speed Limit 20         56         56      0.995      0.982      0.986      0.871
        Speed Limit 30         71         74      0.974      0.946      0.989        0.9
        Speed Limit 40         53         55      0.935      0.945       0.99      0.879
        Speed Limit 50         68         71       0.97      0.899       0.97      0.868
        Speed Limit 60         76         76      0.989      0.908      0.962      0.877
        Speed Limit 7

In [41]:
!ls /content/drive/MyDrive/yolo/adversarial_data_set_fgsm

ls: cannot access '/content/drive/MyDrive/yolo/adversarial_data_set_fgsm': No such file or directory


In [42]:
!unzip -q /content/drive/MyDrive/yolo/adversarial_data_set_fgsm.zip -d /content/


replace /content/content/adversarial_dataset/labels/road466_png.rf.e1995d1db602b4fe4f26c9a2f882bf06.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [43]:
# prompt: save the model

# Save the trained model to Google Drive
model_save_path = '/content/yolov8n_TrafficSigns_defence1.pt'
model.export(format='onnx', imgsz=640, dynamic=True)
model.save(model_save_path)
print(f"Model saved to: {model_save_path}")


Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 72 layers, 3,008,573 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/train5/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 19, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 169.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 178.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.8/280.8 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 180.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 199.6 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 25.4s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu']
requirements: ⚠️ Restart runtime or rerun command for upda

In [44]:
# Evaluate on the test set
results = model.val(
    data="/content/data.yaml",
    split="test",  # Explicitly specify the test split
    batch=16,      # Adjust based on GPU memory
    imgsz=640,      # Match training image size
    conf= 0.25
)

Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,008,573 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/test/labels... 638 images, 1 backgrounds, 0 corrupt: 100%|██████████| 638/638 [00:00<00:00, 643.18it/s]

val: New cache created: /content/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 40/40 [00:09<00:00,  4.18it/s]


                   all        638        770      0.902      0.895      0.936      0.819
           Green Light         77        110      0.917      0.782       0.89       0.58
             Red Light         71         94      0.855       0.69      0.803      0.521
        Speed Limit 10          2          3      0.736      0.943      0.913      0.789
       Speed Limit 100         45         46      0.933      0.891      0.949       0.87
       Speed Limit 110         21         21      0.884       0.81      0.859      0.781
       Speed Limit 120         40         44      0.804      0.909      0.945      0.866
        Speed Limit 20         46         46      0.946      0.957      0.986      0.918
        Speed Limit 30         60         60      0.855      0.884      0.942      0.884
        Speed Limit 40         51         53      0.944      0.956      0.977      0.875
        Speed Limit 50         47         50      0.899       0.88      0.936      0.864
        Speed Limit 6

In [46]:
# Evaluate on the test set
results = model.val(
    data="/content/adv.yaml",
    split="test",  # Explicitly specify the test split
    batch=16,      # Adjust based on GPU memory
    imgsz=640,      # Match training image size
    conf= 0.25
)

Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


val: Scanning /content/content/adversarial_dataset/labels... 638 images, 1 backgrounds, 0 corrupt: 100%|██████████| 638/638 [00:03<00:00, 201.35it/s]

val: New cache created: /content/content/adversarial_dataset/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 40/40 [00:14<00:00,  2.70it/s]


                   all        638        770      0.911       0.77      0.873      0.753
           Green Light         77        110       0.92      0.524      0.741      0.457
             Red Light         71         94      0.929      0.414      0.658      0.426
        Speed Limit 10          2          3      0.743      0.974       0.83      0.657
       Speed Limit 100         45         46      0.864       0.87       0.92      0.828
       Speed Limit 110         21         21      0.953      0.714      0.855      0.769
       Speed Limit 120         40         44      0.915      0.864      0.918      0.813
        Speed Limit 20         46         46          1      0.907      0.965      0.871
        Speed Limit 30         60         60      0.875      0.833      0.904      0.829
        Speed Limit 40         51         53      0.916       0.82      0.918      0.815
        Speed Limit 50         47         50      0.929       0.68      0.848      0.785
        Speed Limit 6



| Class           | Clean mAP@0.5 | Adv mAP@0.5 | Δ (Drop) | Adv mAP@0.5 after defense | Δ after defense |
|----------------|---------------|-------------|----------|----------------------------|-----------------|
| **all**        | 0.927         | 0.312       | -0.615   | 0.753                      | -0.174          |
| Green Light    | 0.927         | 0.312       | -0.615   | 0.457                      | -0.470          |
| Red Light      | 0.816         | 0.328       | -0.488   | 0.426                      | -0.390          |
| Speed Limit 10 | 0.863         | 0.000       | -0.863   | 0.657                      | -0.206          |
| Speed Limit 20 | 0.913         | 0.522       | -0.391   | 0.871                      | -0.042          |
| Speed Limit 30 | 0.937         | 0.511       | -0.426   | 0.829                      | -0.108          |
| Speed Limit 40 | 0.966         | 0.485       | -0.481   | 0.820                      | -0.146          |
| Speed Limit 50 | 0.916         | 0.295       | -0.621   | 0.803                      | -0.113          |
| Speed Limit 60 | 0.953         | 0.239       | -0.714   | 0.825                      | -0.128          |
| Speed Limit 70 | 0.866         | 0.519       | -0.347   | 0.775                      | -0.091          |
| Speed Limit 80 | 0.947         | 0.440       | -0.507   | 0.826                      | -0.121          |
| Speed Limit 90 | 0.806         | 0.440       | -0.366   | 0.750                      | -0.056          |
| Speed Limit 100| 0.941         | 0.511       | -0.430   | 0.784                      | -0.157          |
| Speed Limit 110| 0.959         | 0.511       | -0.448   | 0.769                      | -0.190          |
| Speed Limit 120| 0.966         | 0.511       | -0.455   | 0.813                      | -0.153          |
| Stop           | 0.990         | 0.510       | -0.480   | 0.892                      | -0.098          |
